#@title #Execute This Cell First! 
#This cell's contents should match those of photon.py



In [139]:
#import photon
import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class Photon:

    def __init__(self, Hcomp=0, Vcomp=0):
        self.alpha = Hcomp
        self.beta  = Vcomp

    # This is for debugging purposes only!
    def toString(self):
        if np.isreal(self.alpha):
            string = str(self.alpha) + "|H> "
        else:
            string = str(self.alpha) + "|H> "
        if np.isreal(self.beta):
            if self.beta >= 0:
                string += "+ " + str(self.beta) + "|V>"
            else:
                string += "- " + str(-self.beta) + "|V>"
        else:
            string += "+ " + str(self.beta) + "|V>"
        return string

    def prepareVacuum(self):
        energyPerMode = 0.5; # in units of hbar*omega
        x0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        self.alpha = complex(x0, y0)
        self.beta  = complex(x1, y1)

    def prepare(self, alpha, beta, avgPhotonNumber):
        if avgPhotonNumber < 0:
            raise InputError()
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = alpha * np.sqrt(avgPhotonNumber) + vac.alpha
        self.beta  = beta  * np.sqrt(avgPhotonNumber) + vac.beta

    def prepareH(self, avgPhotonNumber):
        self.prepare(1, 0, avgPhotonNumber)

    def prepareV(self, avgPhotonNumber):
        self.prepare(0,1, avgPhotonNumber)

    def prepareD(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1/np.sqrt(2), avgPhotonNumber)

    def prepareA(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1/np.sqrt(2), avgPhotonNumber)

    def prepareR(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1j/np.sqrt(2), avgPhotonNumber)

    def prepareL(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1j/np.sqrt(2), avgPhotonNumber)

    def measureHV(self, probDarkCount):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.alpha)**2
        intensityV = abs(self.beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

    def measureDA(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a+b)/np.sqrt(2)
        self.beta  = (a-b)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def measureRL(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a-b*1j)/np.sqrt(2)
        self.beta  = (a+b*1j)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "R"
        if outcome == "V": return "L"
        else: return outcome

    def applyPolarizer(self, theta, phi):
	# Apply a polarizing filter according to the input parameters.
	# theta=0,    phi=0:     H polarizer
	# theta=pi/2, phi=0:     V polarizer
	# theta=pi/4, phi=0:     D polarizer
	# theta=pi/4, phi=pi:    A polarizer
	# theta=pi/4, phi=+pi/2: R polarizer
	# theta=pi/4, phi=-pi/2: L polarizer
        z = np.exp(1j*phi)
        a = self.alpha
        b = self.beta
        self.alpha = a*(1+np.cos(2*theta))/2 + b*np.sin(2*theta)/2*np.conj(z)
        self.beta  = a*np.sin(2*theta)/2*z + b*(1-np.cos(2*theta))/2
        # Now add an extra vacuum component.
        vac = Photon()
        vac.prepareVacuum()
        a = vac.alpha
        b = vac.beta
        self.alpha = self.alpha + a*np.sin(theta)**2 + b*(-np.sin(2*theta)/2)*np.conj(z)
        self.beta  = self.beta  + a*(-np.sin(2*theta)/2)*z + b*np.cos(theta)**2

    def applyUnitaryGate(self, theta, phi, lamb):
        U = [[0,0],[0,0]]
        z1 = np.exp(1j*phi)
        z2 = -np.exp(1j*lamb)
        z3 = np.exp(1j*(lamb+phi))
        U[0][0] = np.cos(theta/2)
        U[1][0] = np.sin(theta/2)*z1
        U[0][1] = np.sin(theta/2)*z2
        U[1][1] = np.cos(theta/2)*z3
        a = self.alpha
        b = self.beta
        self.alpha = U[0][0]*a + U[0][1]*b
        self.beta  = U[1][0]*a + U[1][1]*b

    def applyXGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(np.pi, 0, np.pi)

    def applyYGate(self):
        # Applies the Pauli Y gate
        self.applyUnitaryGate(np.pi, np.pi/2, np.pi/2)

    def applyZGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(0, np.pi, 0)

    def applyHGate(self):
        # Applied the Hadamard (half-wavelength) gate
        self.applyUnitaryGate(np.pi/2, 0, np.pi)

    def applyQGate(self):
        # Applies the SH (quarter-wavelength) gate
        self.applyUnitaryGate(np.pi/2, np.pi/2, np.pi)

    def applyNoisyGate(self, p):
        # This operation acts as a depolarizing channel.
	# p = 0 leaves the photon unchanged.
	# p = 1 yields a completely random photon.
	# 0 < p < 1 yields a partially random photon.
        if p < 0 or p > 1:
            raise InputError
        theta = np.arccos(1 - 2*random.uniform(0,1)*p)
        phi   = p*(2*random.uniform(0,1) - 1)*np.pi
        lamb  = p*(2*random.uniform(0,1) - 1)*np.pi
        self.applyUnitaryGate(theta, phi, lamb)

    def applyAttenuation(self, r):
	# This operation acts as a partially reflecting beam splitter.
	# r = 0 leaves the photon unchanged.
	# r = 1 completely absorbs the photon, leaving a vacuum state.
	# 0 < r < 1 partially attenuates the photon and adds some vacuum.
	# r is the reflectivity.
        if r < 0 or r > 1:
            raise InputError
        t = np.sqrt(1-r*r) # t is the transmissivity.
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = (self.alpha)*t + (vac.alpha)*r
        self.beta  = (self.beta )*t + (vac.beta)*r

In [140]:
n = 100 # number of photons

In [141]:
# Alice --------------------------------------------

# Alice generates the raw key.
keyAlice = ""
for i in range(n): # Iterate over the number of photons.
    # Append a random character ('0' or '1') to the end.
    if random.randint(0,1)==0: # Flip a coin (0 or 1).
        keyAlice += '0'
    else:
        keyAlice += '1'
print("keyAlice = " + keyAlice)

keyAlice = 0000110110111011101100010011101010110011011100110001010111000111101011001111000100011000100011001101


In [142]:
# Alice chooses the encoding basis for each key bit.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisAlice = ""
# TODO: Put your code here.
for i in range(n): 
    if random.randint(0,1)==0: # Flip a coin (0 or 1).
        basisAlice += '+'
    else:
        basisAlice += 'x'
print("basisAlice  = " + basisAlice)

basisAlice  = x++x++x++xx+x++xx+xxx++x++xxx++x+x+x+x++x+x+xxx+x++x+++x+x+x++x++x+xxxx++xxx+++xx+xx+x+xx+x+xx+++x+x


In [143]:
# Alice selects a photon state according to the key and basis.
# This should be a string of the characters 'H', 'V', 'D', 'A'.
photonAlice = ""
# TODO: Put your code here.
for i in range(n): # Iterate over the number of qubits.
    if basisAlice[i]== '+': 
        if keyAlice[i]=='0':
          photonAlice+='H'
        else:
          photonAlice+='V'
    else:
        if keyAlice[i]=='0':
          photonAlice+='D'
        else:
          photonAlice+='A'

print("photonAlice  = " + photonAlice)

photonAlice  = DHHDVVDVVDAVAHVAAHAADHHAHHAAAHVDVDVAHDVVDVAVDDAVDHHAHVHAVAHDHVAVVDVDAADHVAAAHHHADHDAVDHDAHDHAAHHVAHA


In [144]:
# Alice prepares and sends each photon.
# Use the methods of the Photon class to prepare each photon.
photonArray = [Photon() for i in range(n)]
# TODO: Put your code here.
for j in range(n):
  tempPhoton = Photon()
  if photonAlice[j]=='A':
    tempPhoton.prepareA(1)
    print("A Prepared")
  elif photonAlice[j]=='D':
    tempPhoton.prepareD(1)
    print("D Prepared")
  elif photonAlice[j]=='H':
    tempPhoton.prepareH(1)
    print("H Prepared")
  elif photonAlice[j]=='V':
    tempPhoton.prepareV(1)
    print("V Prepared")
  photonArray[j] = tempPhoton

D Prepared
H Prepared
H Prepared
D Prepared
V Prepared
V Prepared
D Prepared
V Prepared
V Prepared
D Prepared
A Prepared
V Prepared
A Prepared
H Prepared
V Prepared
A Prepared
A Prepared
H Prepared
A Prepared
A Prepared
D Prepared
H Prepared
H Prepared
A Prepared
H Prepared
H Prepared
A Prepared
A Prepared
A Prepared
H Prepared
V Prepared
D Prepared
V Prepared
D Prepared
V Prepared
A Prepared
H Prepared
D Prepared
V Prepared
V Prepared
D Prepared
V Prepared
A Prepared
V Prepared
D Prepared
D Prepared
A Prepared
V Prepared
D Prepared
H Prepared
H Prepared
A Prepared
H Prepared
V Prepared
H Prepared
A Prepared
V Prepared
A Prepared
H Prepared
D Prepared
H Prepared
V Prepared
A Prepared
V Prepared
V Prepared
D Prepared
V Prepared
D Prepared
A Prepared
A Prepared
D Prepared
H Prepared
V Prepared
A Prepared
A Prepared
A Prepared
H Prepared
H Prepared
H Prepared
A Prepared
D Prepared
H Prepared
D Prepared
A Prepared
V Prepared
D Prepared
H Prepared
D Prepared
A Prepared
H Prepared
D Prepared

In [145]:
# Eve   --------------------------------------------

# You should implement this section after completing Alice and Bob.
# Eve is allowed to do whatever she wants to the photonAlice array.
# She cannot, however, have knowledge of Alice's or Bob's choice of bases, 
# nor Bob's measurement outcomes, until they are publicly announced.

In [146]:
# Eve selects a subsample of photons from Alice to measure.
# interceptIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=intercepted
interceptIndex = ""
# TODO: Put your code here.
for i in range(n): # Iterate over the number of qubits.
    if random.randint(0,1)==0: # Flip a coin (0 or 1).
        interceptIndex += '0'
    else:
        interceptIndex += '1'
print("interceptIndex  = " + interceptIndex)

interceptIndex  = 0001010100000001110010001111010001100011000010010101010000001010000101000101010111110100111111011101


In [147]:
# Eve chooses a basis to measure each intercepted photon.
# basisEve should be a string of n characters.
# Use the convention '+'=H/V, 'x'=D/A, ' '=not measured
basisEve = ""
# TODO: Put your code here.
for i in range(n): 
    if random.randint(0,1)==0:
        basisEve += '+'
    else:
        basisEve += 'x'
print("basisEve    = " + basisEve)

basisEve    = ++++xx+x++x++xxx++++xxxxx++x+x+xx+xxxxx+x+++xxx++x+x+x++xxxx+x+++x+++xxxxx+xx+x++x++++x++xx+xx++xx++


In [148]:
# Eve performs a measurement on each photon.
# outcomeEve should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeEve = ""
for k in range(n): 
  if basisEve[k]=='x':
    tempPhoton = photonArray[k]
    tempStr = tempPhoton.measureDA(0.9) 
    #print(tempPhoton.toString())
    if (tempStr!='M' and tempStr!='N'):
      outcomeEve+=str(tempStr)
    else:
      outcomeEve+=' '
  elif basisEve[k]=='+':
    tempPhoton = photonArray[k]
    tempStr = tempPhoton.measureHV(0.9)
    if (tempStr!='M' and tempStr!='N'):
      outcomeEve+=str(tempStr)
    else:
      outcomeEve+=' '
print("outcomeEve  = " + outcomeEve)
# TODO: Put your code here.

outcomeEve  =  HV  DV   A              H      D D A  V V             V       H     DDA   A H         V  D       H 


In [149]:
# Eve resends photons to Bob.
# Be sure to handle the cases in which Eve gets an invalid measurement.
# TODO: Put your code here.
for k in range(n):
    if outcomeEve[k]==' ':
        photonArray[k].prepareVacuum()
    elif outcomeEve[k]=='H':
        photonArray[k].prepareH(1)
    elif outcomeEve[k]=='V':
        photonArray[k].prepareV(1)
    elif outcomeEve[k]=='D':
        photonArray[k].prepareD(1)
    elif outcomeEve[k]=='A':
        photonArray[k].prepareA(1)
    else:
        photonArray[k].prepareVacuum()
        print("Error in Eve's Resend")

In [150]:
# OPTIONAL: Put any other nasty tricks here.
for k in range(n):
    if outcomeEve[k]==' ':
        photonArray[k].applyAttenuation(0.9)
    else:
        photonArray[k].applyAttenuation(0.1)
    #print(photonArray[k].toString())

In [151]:
# Bob   --------------------------------------------

# Bob chooses a basis to measure each photon.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisBob = ""
# TODO: Put your code here.
for i in range(n): 
    if random.randint(0,1)==0: # Flip a coin (0 or 1).
        basisBob += '+'
    else:
        basisBob += 'x'
print("basisBob    = " + basisBob)

basisBob    = x++++x+x+x++x+++xxxx++++++x+xxxxx+x++xx++x+++x++x+x+xx+++xx++++x++x+xx+x++x+x+xxx+++xx+++++xx+x++xxx


In [152]:
# Bob performs a measurement on each photon.
# Use the methods of the Photon class to measure each photon.
# outcomeBob should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeBob = ""
for k in range(n): 
  if basisBob[k]=='x':
    tempPhoton = photonArray[k]
    tempStr = tempPhoton.measureDA(0.9) 
    #print(tempPhoton.toString())
    if (tempStr!='M' and tempStr!='N'):
      outcomeBob+=str(tempStr)
    else:
      outcomeBob+=' '
  elif basisBob[k]=='+':
    tempPhoton = photonArray[k]
    tempStr = tempPhoton.measureHV(0.9)
    if (tempStr!='M' and tempStr!='N'):
      outcomeBob+=str(tempStr)
    else:
      outcomeBob+=' '

print("outcomeBob  = " + outcomeBob)
# TODO: Put your code here.

outcomeBob  =   V  D A  H D VVAD D V     H A  DV V   VV V   H  HDV  H  A  H H   DH   AHH   H      A   H H AVD   D 


In [153]:
# Bob infers the raw key.
# keyBob should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement
keyBob = ""
# TODO: Put your code here.
for i in range(n):
  if basisBob[i]=='+':
    if outcomeBob[i]=='H':
      keyBob+='0'
    elif outcomeBob[i]=='V':
      keyBob+='1'
    elif outcomeBob[i]=='D':
      keyBob+=str(random.randint(0,1))
    elif outcomeBob[i]=='A':
      keyBob+=str(random.randint(0,1))
    elif outcomeBob[i]==' ':
      keyBob+='-'
  else:
    if outcomeBob[i]=='D':
      keyBob+='0'
    elif outcomeBob[i]=='A':
      keyBob+='1'
    elif outcomeBob[i]=='H':
      keyBob+=str(random.randint(0,1))
    elif outcomeBob[i]=='V':
      keyBob+=str(random.randint(0,1))
    elif outcomeBob[i]==' ':
      keyBob+='-'

print("keyBob      = " + keyBob)

keyBob      = --1--0-1--0-0-1110-0-1-----0-1--01-1---11-1---0--001--0--1--0-0---00---100---0------1---0-0-110---0-


In [154]:
# -----------------------------------------------------------
# Alice and Bob now publicly announce which bases they chose.
# Bob also announces which of his measurements were invalid.
# -----------------------------------------------------------

In [155]:
# Alice & Bob ----------------------------------------------------------

# Alice and Bob extract their sifted keys.
# siftedAlice and siftedBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
siftedAlice = ""
siftedBob   = ""
# TODO: Put your code here.

for i in range(n):
  if ((basisBob[i]==basisAlice[i])):
    siftedAlice+=keyAlice[i]
  else:
    siftedAlice+= " "

for i in range(n):
  if (keyBob[i]!='-'):
    siftedBob+=keyBob[i]
  else:
    siftedBob+=' '
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)

siftedAlice = 000 1   10 1101 1 11 00 001 1  0    00 1   1 0 100    0 11  01  1   11  1 1  0 100   00  0  1  011 1
siftedBob   =   1  0 1  0 0 1110 0 1     0 1  01 1   11 1   0  001  0  1  0 0   00   100   0      1   0 0 110   0 


In [156]:
# Alice and Bob use a portion of their sifted keys to estimate the quantum bit error rate (QBER).
# sampleIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=sampled
# The QBER is the fraction of mismatches within the sampled portion.
# For large samples, it should be close to the actual QBER,
# which Alice and Bob, of course, do not know.
sampleIndex = ""
sampledBobQBER = 0
numDiff = 0
count = 0
# TODO: Put your code here.
for i in range(len(siftedAlice)):
  if (i % 2 == 0) :
      sampleIndex += "1"
      if (siftedAlice[i] != siftedBob[i]):
        numDiff += 1
  else :
    sampleIndex += "0"
sampledBobQBER = numDiff / len(siftedAlice) / 2
print(sampledBobQBER)

0.165


In [157]:
# Alice and Bob remove the portion of their sifted keys that was sampled.
# Since a portion of the sifted key was publicly revealed, it cannot be used.
# secureAlice and secureBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
secureAlice = ""
secureBob = ""
# TODO: Put your code here.
for i in range(len(siftedAlice)):
  if (i % 2 != 0) :
    secureAlice += siftedAlice[i]
    secureBob += siftedBob[i]
  else :
    secureAlice += " "
    secureBob += " "

print(secureAlice)
print(secureBob)

 0       0 1 0     1 0   0     0     0 1   1 0 1 0       1   1       1       0 1 0   0   0     0 1 1
     0 1       1 0 0 1     0 1   1 1   1         0 1     1         0   1 0   0               1      


In [158]:
# Alice and Bob make a hard determination whether the channel is secure.
# If it looks like there's something fishy, better hit the kill switch!
channelSecure = True # default value, to be changed to False if Eve suspected
# TODO: Put your code here.
if (sampledBobQBER > .3) :
  channelSecure = False
print(channelSecure)

True


In [159]:
# Eve ------------------------------------------------------------------

In [160]:
# Eve infers the raw key.
# keyEve should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement, ' '=not measured
keyEve = ""
# TODO: Put your code here.
for i in range(n):
    keyEve+=outcomeEve[i]
print("keyEve      = " + keyEve)

keyEve      =  HV  DV   A              H      D D A  V V             V       H     DDA   A H         V  D       H 


In [161]:
# Eve extracts her sifted key.
# Knowing what Alice and Bob have publically revealed, Eve
# now selects which portion of her sifted key to keep.
# stolenEve should be strings of length n.
# Use the '0', '1', ' '=removed
stolenEve = ""
# TODO: Put your code here.
for i in range(n):
    #IndexError: string index out of range
    if (basisAlice[i]==basisEve[i]):
        if (keyEve[i]!='-'):
            stolenEve+=keyEve[i]
        else:
            stolenEve+=' '
    else:
        stolenEve+=' '
print("stolenEve   = " + stolenEve)

stolenEve   =  HV       A              H             V V                     H     DD    A H            D       H 


In [162]:
# ANALYSIS -------------------------------------------------------------

# Below is a standard set of metrics to evaluate each protocol.
# You need not change any of what follows.

# Compare Alice and Bob's sifted keys.
numMatchBob = 0
actualBobQBER = 0
secureKeyRateBob = 0
secureKeyLengthBob = 0
for i in range(len(secureAlice)):
    if secureAlice[i] != ' ':
       secureKeyLengthBob += 1
       if secureAlice[i] == secureBob[i]:
           numMatchBob += 1

# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1-actualBobQBER) * secureKeyLengthBob / n * 1e6;

# Compare Alice and Eve's sifted keys.
numMatchEve = 0
actualEveQBER = 0
stolenKeyRateEve = 0
stolenKeyLengthEve = 0
for i in range(len(stolenEve)):
    if stolenEve[i] != ' ':
       stolenKeyLengthEve += 1
       if secureAlice[i] == stolenEve[i]:
           numMatchEve += 1
# Compute the actual quantum bit error rate for Eve.
if stolenKeyLengthEve > 0:
    actualEveQBER = 1 - numMatchEve / stolenKeyLengthEve
else:
    actualEveQBER = float('nan')
# Compute the stolen key rate, assuming each trial takes 1 microsecond.
stolenKeyRateEve = (1-actualEveQBER) * stolenKeyLengthEve / n * 1e6;


# DISPLAY RESULTS ------------------------------------------------------

print("")
print("basisAlice  = " + basisAlice)
print("basisBob    = " + basisBob)
print("basisEve    = " + basisEve)
print("")
print("keyAlice    = " + keyAlice)
print("keyBob      = " + keyBob)
print("keyEve      = " + keyEve)
print("")
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)
print("")
print("secureAlice = " + secureAlice)
print("secureBob   = " + secureBob)
print("stolenEve   = " + stolenEve)
print("")
if not channelSecure:
    secureKeyRateBob = 0;
    stolenKeyRateEve = 0;
    print("*********************************************")
    print("* ALERT! The quantum channel is not secure. *")
    print("*********************************************")
    print("")
print("sampledBobQBER = " + str(sampledBobQBER))
print("actualBobQBER  = " + str(actualBobQBER))
print("actualEveQBER  = " + str(actualEveQBER))
print("")
print("secureKeyRateBob = " + str(secureKeyRateBob/1000) + " kbps")
print("stolenKeyRateEve = " + str(stolenKeyRateEve/1000) + " kbps")

# Your goal is to maximize secureKeyRateBob and minimize stolenKeyRateEve.


basisAlice  = x++x++x++xx+x++xx+xxx++x++xxx++x+x+x+x++x+x+xxx+x++x+++x+x+x++x++x+xxxx++xxx+++xx+xx+x+xx+x+xx+++x+x
basisBob    = x++++x+x+x++x+++xxxx++++++x+xxxxx+x++xx++x+++x++x+x+xx+++xx++++x++x+xx+x++x+x+xxx+++xx+++++xx+x++xxx
basisEve    = ++++xx+x++x++xxx++++xxxxx++x+x+xx+xxxxx+x+++xxx++x+x+x++xxxx+x+++x+++xxxxx+xx+x++x++++x++xx+xx++xx++

keyAlice    = 0000110110111011101100010011101010110011011100110001010111000111101011001111000100011000100011001101
keyBob      = --1--0-1--0-0-1110-0-1-----0-1--01-1---11-1---0--001--0--1--0-0---00---100---0------1---0-0-110---0-
keyEve      =  HV  DV   A              H      D D A  V V             V       H     DDA   A H         V  D       H 

siftedAlice = 000 1   10 1101 1 11 00 001 1  0    00 1   1 0 100    0 11  01  1   11  1 1  0 100   00  0  1  011 1
siftedBob   =   1  0 1  0 0 1110 0 1     0 1  01 1   11 1   0  001  0  1  0 0   00   100   0      1   0 0 110   0 

secureAlice =  0       0 1 0     1 0   0     0     0 1   1 0 1 0       1   1